In [1]:
import os
import imageio
from skimage.io import imread_collection

import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow.keras.layers as layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

import statsmodels.api as sm

In [2]:
## directories
paths = ['images',
            'images/1dim_uni_to_normal',
                'images/1dim_uni_to_normal/gif',
                'images/1dim_uni_to_normal/final',
        ]

paths_1 = ['images',
            'images/1dim_norm(0,8)_to_normal',
        ]

paths_2 = ['images',
            'images/1dim_norm(8,1)_to_normal',
                'images/1dim_norm(8,1)_to_normal/gif',
                'images/1dim_norm(8,1)_to_normal/final',
        ]

for i in paths:
    if not os.path.exists(i):
        os.makedirs(i)
        
for i in paths_1:
    if not os.path.exists(i):
        os.makedirs(i)

for i in paths_2:
    if not os.path.exists(i):
        os.makedirs(i)

In [2]:
def generate_noise(samples, dimensions=2):
    return np.random.uniform(-1,1, (samples, dimensions))
    #return np.random.normal(0,8, (samples,dimensions))
    #return np.random.normal(8,1, (samples,dimensions))
    
def generate_data(samples, dimensions=2):
    return np.random.normal(0,1, (samples, dimensions))

# mapping (R,R)-->([-1,1], [-1,1])
def generator(d, output_dim):
    input_layer = layers.Input((d, ))
    X = input_layer
    # NN with 3 dense layers
    for i in range(4):
        X = layers.Dense(16)(X)
        X = layers.LeakyReLU(0.1)(X)
    output_layer = layers.Dense(output_dim)(X)
    G = Model(input_layer, output_layer)
    return G

# mapping (R,R)-->[0,1]
def discriminator(dim):
    input_layer = layers.Input((dim,))
    X = input_layer
    # NN with 1 dense layer
    for i in range(2):
        X = layers.Dense(64)(X)
        X = layers.LeakyReLU(0.1)(X)
    output_layer = layers.Dense(1, activation='sigmoid')(X)
    D = Model(input_layer, output_layer)
    D.compile(Adam(learning_rate = 0.002, beta_1 = 0.5),
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])
    return D

# build GAN given a discriminator and a generator
def GAN(G, D, d):
    D.trainable = False
    input_layer = layers.Input((d,))
    X = G(input_layer)
    output_layer = D(X)
    GAN = Model(input_layer, output_layer)
    GAN.compile(Adam(learning_rate = 0.001, beta_1 = 0.5),
                       loss = 'binary_crossentropy',
                       metrics = ['accuracy'])
    return GAN
    

In [3]:
# VISUALIZATION

def plots(G, D, step, D_loss, G_loss, filename):
    file, ax = plt.subplots(2, 2, figsize=(8,8))
    
    # plot losses
    ax[0,0].plot(step, G_loss, label='G loss', 
                    c='darkgreen', zorder=50, alpha=0.8,)
    ax[0,0].plot(step, D_loss, label='D loss',
                    c='darkblue', zorder=55, alpha=0.8,)
    ax[0,0].set_xlim(0, max(max(step), batches)+5)
    #ax[0,0].set_ylim(0.45, 1.0)
    ax[0,0].set_xlabel('Epoch')
    ax[0,0].legend(loc=1, frameon=False)
    
    # plot real and generated samples
    fake_samples = G.predict(test_noise, batch_size=len(test_noise))
    x_val = np.linspace(-3, 3, 301)
    y_val = stats.norm(0,1).pdf(x_val)
    ax[0,1].plot(x_val, y_val, color='k', label='real')
    ax[0,1].fill_between(x_val, np.zeros(len(x_val)), y_val, color='k', alpha=0.6)
    sns.kdeplot(fake_samples.flatten(), color='green', alpha=0.6, label='GAN', ax=ax[0,1], shade=True)
    #ax[0,1].set_xlim(-3,3)
    ax[0,1].legend(loc=1, frameon=False)
    ax[0,1].set_xlabel('Sample Space')
    ax[0,1].set_ylabel('Density')
    
    # plot confidence of the discriminator
    confi = D.predict(grid_sample, batch_size=bs).flatten()
    ax[1,0].plot(grid_sample.flatten(), confi, c='b')
    lower, upper = -3, 3
    for i in range(0, len(confi), 50):
        if i==0:
            continue
        ax[1,0].plot([(i/len(confi))*(upper-lower) + lower, ]*2,
                        [0, confi[i]], c='b')
    ax[1,0].fill_between(grid_sample.flatten(), np.zeros(len(confi)), confi, color='b', alpha=0.6)
    ax[1,0].set_xlabel('Sample Space')
    ax[1,0].set_ylabel('Discriminator Value')
    ax[1,0].set_xlim(lower,upper)
    ax[1,0].set_ylim(0.0,1.0)
        
    # Q-Q Plot
    qq = fake_samples.reshape(len(fake_samples,))
    sm.qqplot(qq, line='45', markerfacecolor='cornflowerblue', markeredgecolor='cornflowerblue',  ax=ax[1,1])
    
    plt.tight_layout()
    plt.savefig(filename, transparent=True)
    plt.close()
    return

In [4]:
# parameters
# latent dimension
d = 1

# subsample for gif, set to zero if no gif wanted
batches = 500

# batch size
bs = 512

# plot frequency
freq = 1

test_noise = generate_noise(5000, d)
test_samples = generate_data(5000, d)

G = generator(d, 1)
D = discriminator(1)
GAN = GAN(G, D, d)

grid_sample = np.linspace(-3, 3, 603)[1:-1].reshape((-1, 1))

In [5]:
step = []
D_acc = []
G_acc = []
D_loss = []
G_loss = []
count = []

In [6]:
# produce pictures for GIF

for k in range(batches):
    print(f'step: {k}/{batches}', end='\r')
    
    # train discriminator
    D.trainable = True
    real_data = generate_data(bs//2,d)
    fake_data = G.predict(generate_noise(bs//2, d), batch_size=bs//2)
    data = np.concatenate((real_data, fake_data), axis=0)
    
    real_labels = np.ones((bs//2, 1))
    fake_labels = np.zeros((bs//2,1))
    labels = np.concatenate((real_labels, fake_labels), axis=0)
    
    _D_loss, _D_acc = D.train_on_batch(data, labels)
    
    # train generator
    D.trainable = False
    noise = generate_noise(bs, d)
    labels = np.ones((bs, 1))
    _G_loss, _G_acc = GAN.train_on_batch(noise, labels)
    
    if k % freq == 0:
        step.append(k)
        D_loss.append(_D_loss)
        G_loss.append(_G_loss)
        plots(G=G, D=D, step=step, D_loss=D_loss, G_loss=G_loss,
                #filename=f'images/1dim_uni_to_normal/gif/pic.{k:03d}.jpeg')
                filename=f'images/1dim_norm(8,1)_to_normal/gif/pic.{k:03d}.jpeg')
print()

step: 499/500


In [7]:
#make GIF
#directory = 'images/1dim_uni_to_normal/gif/*.jpeg'
directory = 'images/1dim_norm(8,1)_to_normal/gif/*.jpeg'
images = imread_collection(directory)
imageio.mimsave('images/1dim_norm(8,1)_to_normal/final/animation3.mp4', images, quality=9)

In [8]:
total_steps = 10000

for k in range(batches, total_steps):
#for k in range(6000, total_steps):
    print(f'step: {k}/{total_steps}', end='\r')
    
    # train discriminator
    D.trainable = True
    real_data = generate_data(bs//2,d)
    fake_data = G.predict(generate_noise(bs//2, d), batch_size=bs//2)
    data = np.concatenate((real_data, fake_data), axis=0)
    
    real_labels = np.ones((bs//2, 1))
    fake_labels = np.zeros((bs//2,1))
    labels = np.concatenate((real_labels, fake_labels), axis=0)
    
    _D_loss, _D_acc = D.train_on_batch(data, labels)
    
    # train generator
    D.trainable = False
    noise = generate_noise(bs, d)
    labels = np.ones((bs, 1))
    _G_loss, _G_acc = GAN.train_on_batch(noise, labels)    
    
    if k % freq == 0:
        step.append(k)
        D_loss.append(_D_loss)
        G_loss.append(_G_loss)
        
#plots(G=G, D=D, step=step, D_loss=D_loss, G_loss=G_loss, filename=f'images/1dim_uni_to_normal/final/pic.{k:03d}_4.png')
#plots(G=G, D=D, step=step, D_loss=D_loss, G_loss=G_loss, filename=f'images/1dim_norm(0,8)_to_normal/pic.{k:03d}.png')
plots(G=G, D=D, step=step, D_loss=D_loss, G_loss=G_loss, filename=f'images/1dim_norm(8,1)_to_normal/pic.{k:03d}_3.png')